# __Brough Lecture Notes: GARCH Models - Asymmetric Models__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on several chapters in the series of books on [Market Risk Analysis](http://carolalexander.org/market-risk-analysis/) by Carol Alexander.

<br>

There is a mind-numbing alphabet soup of variations on GARCH models. These notes introduce you to one simple extension known as the GJR-GARCH model, named for the initials of its [inventors](https://bit.ly/2U1d8Kj).

<br>

Ever since [Black 1976](https://bit.ly/2CGBNck) economists have become aware of what Black called the _leverage effect_.
This basically means that there is an _asymmetric_ response to past volatility shocks.

<br>

To account for this GJR add a single extra _leverage_ parameter that augments the weight of the response in the model to negative market shocks. 

<br>

The model can be written down as: 

<br>

$$
\sigma_{t}^{2} = \omega + \alpha \epsilon_{t-1}^{2} + \lambda I_{\{\epsilon_{t-1} < 0\}} \epsilon_{t-1}^{2} + \beta \sigma_{t-1}^{2}
$$

<br>

where $I_{\{\epsilon_{t-1} < 0 \}} = 1$ if $\epsilon_{t-1} < 0$ and $0$ otherwise (called an indicator function).

<br>

MLE parameter estimation is based on the usual GARCH likelihood function, where $\sigma_{t}$ depends now on $\lambda$.

<br>

__NB:__ Note that if $\lambda = 0$ then we have a simple GARCH(1,1) model, so the GJR-GARCH model is a generalization
        of the standard GARCH model. 
        
<br>

__NB:__ Note that a general GJR-GARCH(p, o, q) model could be written down similar to a GARCH(p, q) model where the $o$
        parameter is the number of lags in the _leverage term_. 
        
<br>

The GJR-GARCH model has often been found to improve the fit to data relative to a standard symmetric GARCH model. 

<br>

The standard model specification is a GRJ-GARCH(1,1,1) model (similar to the GARCH(1,1) case). 

## Estimation via MLE

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

import seaborn
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv

import scipy.optimize as opt

In [57]:
def gjr_garch_likelihood(parameters, data, sigma2, out=None):
    '''Returns negative log-likelihood for GJR-GARCH(1,1,1) model.'''
    mu = parameters[0]
    omega = parameters[1]
    alpha = parameters[2]
    gamma = parameters[3]
    beta = parameters[4]
    
    T = size(data,0)
    eps = data - mu
    # Data and sigma2 are T by 1 vectors
    for t in range(1,T):
        sigma2[t] = (omega + alpha * eps[t-1]**2 + gamma * eps[t-1]**2 * (eps[t-1]<0) + beta * sigma2[t-1])
        
    logliks = 0.5*(log(2*pi) + log(sigma2) + eps**2/sigma2)
    loglik = sum(logliks)
    
    if out is None:
        return loglik
    else:
        return loglik, logliks, copy(sigma2)

In [58]:
def gjr_constraint(parameters, data, sigma2, out=None):
    '''Constraint that alpha+gamma/2+beta<=1'''
    
    alpha = parameters[2]
    gamma = parameters[3]
    beta = parameters[4]
    
    return array([1-alpha-gamma/2-beta])

In [59]:
def hessian_2sided(fun, theta, args):
    f = fun(theta, *args)
    h = 1e-5*np.abs(theta)
    thetah = theta + h
    h = thetah - theta
    K = size(theta,0)
    h = np.diag(h)
    
    fp = zeros(K)
    fm = zeros(K)
    for i in range(K):
        fp[i] = fun(theta+h[i], *args)
        fm[i] = fun(theta-h[i], *args)
        
        
    fpp = zeros((K,K))
    fmm = zeros((K,K))
    for i in range(K):
        for j in range(i,K):
            fpp[i,j] = fun(theta + h[i] + h[j], *args)
            fpp[j,i] = fpp[i,j]
            fmm[i,j] = fun(theta - h[i] - h[j], *args)
            fmm[j,i] = fmm[i,j]
            
    hh = (diag(h))
    hh = hh.reshape((K,1))
    hh = hh @ hh.T
    
    H = zeros((K,K))
    
    for i in range(K):
        for j in range(i,K):
            H[i,j] = (fpp[i,j] - fp[i] - fp[j] + f + f - fm[i] - fm[j] + fmm[i,j])/hh[i,j]/2
            H[j,i] = H[i,j]
            
    return H

In [60]:
ibm = pd.read_csv("./data/IBM-1999-2003.csv", parse_dates=True, index_col=1)

In [61]:
ibm.head()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
1999-01-04,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,183.0000,-0.007458,2,-0.007458,-0.000919
1999-01-05,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,189.6250,0.036202,2,0.036202,0.013582
1999-01-06,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,188.7500,-0.004614,2,-0.004614,0.022140
1999-01-07,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,190.1875,0.007616,2,0.007616,-0.002051
1999-01-08,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,187.5625,-0.013802,2,-0.013802,0.004221


In [62]:
ibm.tail()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
2003-12-24,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.27,-0.005604,1,-0.005604,-0.001807
2003-12-26,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.90,0.006828,1,0.006828,0.001691
2003-12-29,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,93.52,0.006674,1,0.006674,0.012401
2003-12-30,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.63,-0.009517,1,-0.009517,0.000144
2003-12-31,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.68,0.000540,1,0.000540,0.002055


In [63]:
prc = np.abs(ibm.PRC.values)
ret = 100 * ibm.RET.values

In [64]:
ret.mean()

0.03420429936305733

In [65]:
ret.var()

6.305847511637566

In [66]:
startingVals = array([ret.mean(), ret.var() * .01, 0.03, 0.09, .09])

In [67]:
finfo = np.finfo(np.float64)
bounds = [(-10*ret.mean(), 10*ret.mean()), (finfo.eps, 2*ret.var()), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

In [68]:
T = ret.shape[0]

In [69]:
sigma2 = np.ones(T) * ret.var()
args = (np.asarray(ret), sigma2)

In [70]:
results = opt.minimize(gjr_garch_likelihood, x0=startingVals, args=args, method="SLSQP", bounds=bounds)

In [71]:
results

     fun: 2838.8729139578736
     jac: array([-0.0020752 , -0.0279541 , -0.04510498, -0.02459717, -0.08877563])
 message: 'Optimization terminated successfully.'
    nfev: 247
     nit: 31
    njev: 31
  status: 0
 success: True
       x: array([0.02119785, 0.01214893, 0.00318755, 0.08323103, 0.95706644])

In [72]:
results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='Nelder-Mead') #, bounds=bounds)
results

/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


 final_simplex: (array([[0.02113193, 0.01216813, 0.00318826, 0.08324996, 0.95705696],
       [0.02109723, 0.01216096, 0.00318452, 0.08322738, 0.95707071],
       [0.02119908, 0.01213519, 0.00318713, 0.0832479 , 0.95706285],
       [0.02115491, 0.01215956, 0.00318295, 0.08325718, 0.95705829],
       [0.02118094, 0.01217498, 0.00318836, 0.08321967, 0.95706331],
       [0.02107593, 0.01216487, 0.00319355, 0.08323139, 0.95706008]]), array([2838.87291525, 2838.8729159 , 2838.8729161 , 2838.87291614,
       2838.87291706, 2838.87291715]))
           fun: 2838.8729152505502
       message: 'Optimization terminated successfully.'
          nfev: 929
           nit: 578
        status: 0
       success: True
             x: array([0.02113193, 0.01216813, 0.00318826, 0.08324996, 0.95705696])

In [73]:
results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='BFGS') #, bounds=bounds)
results

/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/home/brough/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app
/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app
/home/brough/anaconda3/l

      fun: nan
 hess_inv: array([[ 0.99937759, -0.00400756, -0.00388361, -0.00206535, -0.02430576],
       [-0.00400756,  0.49920941, -0.50221567, -0.09995112,  0.01599693],
       [-0.00388361, -0.50221567,  0.54174152, -0.08650789, -0.00544494],
       [-0.00206535, -0.09995112, -0.08650789,  0.981314  , -0.05719997],
       [-0.02430576,  0.01599693, -0.00544494, -0.05719997,  0.00435246]])
      jac: array([nan, nan, nan, nan, nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 826
      nit: 3
     njev: 117
   status: 2
  success: False
        x: array([ -17.073305  ,  926.1137369 , -886.8564918 , -406.47181504,
         42.00065192])

In [74]:
results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='L-BFGS-B', bounds=bounds)
results

      fun: 2887.5831066793753
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 22.33819032, 165.98751245, 675.31263994, 210.4547093 ,
       964.62335932])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 300
      nit: 21
   status: 0
  success: True
        x: array([0.15079003, 0.19780451, 0.13782738, 0.01112935, 0.87147362])

## DEM2GBP GJR-GARCH(1,1,1) Model

In [75]:
df = pd.read_csv("./data/dem2gbp.csv")

In [76]:
#df.head()
#df.tail()

In [77]:
r = df.DEM2GBP.values[:749] 
sigma = np.ones_like(r) * r.var()
args = (r, sigma)

startingVals = array([ret.mean(), ret.var() * .01, 0.03, 0.09, .09])
finfo = np.finfo(np.float64)
bounds = [(-10*ret.mean(), 10*ret.mean()), (finfo.eps, 2*ret.var()), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='SLSQP', bounds=bounds) #, constraints=garch_constraints)

In [78]:
results

     fun: 578.6397173669247
     jac: array([-0.00814056, -0.02651215, -0.03795624, -0.0211792 , -0.03391266])
 message: 'Optimization terminated successfully.'
    nfev: 163
     nit: 20
    njev: 20
  status: 0
 success: True
       x: array([-0.03391969,  0.05330952,  0.142602  ,  0.16036624,  0.61058383])

In [79]:
results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='L-BFGS-B', bounds=bounds)
results

      fun: 578.6397173369253
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.25055521e-04, -3.63797881e-04, -4.54747351e-05, -4.54747351e-05,
       -5.68434189e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 210
      nit: 24
   status: 0
  success: True
        x: array([-0.03391946,  0.05330661,  0.14259833,  0.16035524,  0.61059889])

In [80]:
results = opt.minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='Nelder-Mead')
results

 final_simplex: (array([[-0.03392576,  0.05330526,  0.14256422,  0.16044095,  0.61059661],
       [-0.03390515,  0.05331597,  0.14261052,  0.16043911,  0.6105243 ],
       [-0.03389737,  0.05331161,  0.14257417,  0.16038122,  0.61059634],
       [-0.03391434,  0.05332485,  0.14262952,  0.16034586,  0.61051756],
       [-0.03392366,  0.05331848,  0.14256942,  0.16048756,  0.61055192],
       [-0.03389717,  0.05331908,  0.14254131,  0.16036256,  0.61056078]]), array([578.63971815, 578.63971859, 578.6397186 , 578.63971863,
       578.63971923, 578.63972106]))
           fun: 578.6397181543871
       message: 'Optimization terminated successfully.'
          nfev: 547
           nit: 335
        status: 0
       success: True
             x: array([-0.03392576,  0.05330526,  0.14256422,  0.16044095,  0.61059661])